In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import pandas as pd
from bs4 import BeautifulSoup
import json

url = url = "http://academico.unemat.br/"


def site_login(url):

    option = Options()
    option.headless = True
    driver = webdriver.Firefox(options=option)

    login = str(input("Digite seu login: "))
    password = str(input("Digite sua senha: "))    
    
    driver.get(url)
    print("Waiting for connection: ", end="")
    driver.find_element_by_id("uid").send_keys(login)
    driver.find_element_by_id("pwd").send_keys(password)
    driver.find_element_by_id("btnLogin").click()
    driver.find_element_by_id("m19").click()
    driver.find_element_by_id("m19").click()
    element = driver.find_element_by_id("tbodym46")
    html_content = element.get_attribute("outerHTML")

    # Parse HTML (Parsear o conteúdo HTML) - BeaultifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')
    table = soup.find(name='table')

    # Data Structure Conversion (Estruturar conteúdo em um Data Frame) - Pandas
    df_full = pd.read_html(str(table))[0]

    driver.quit()

    return(df_full)


def process_df(df):
    cr, dem = 0, 1
    df = df.drop(columns=[1])
    df.index = df.index.astype(str, copy=False)
    df = df.drop("0")
    df = df.rename(columns={
        0: "Período",
        2: "Disciplina",
        3: "Professor",
        4: "Carga Horária",
        5: "Nota",
        6: "Frequência",
        7: "Estado",
        8: "Créditos"})
    for index, row in df.iterrows():
        df["Disciplina"][index] = df["Disciplina"][index].split("(")[0]
        df["Professor"][index] = " ".join(
            df["Professor"][index].split(" ")[3:])
        df["Créditos"][index] = int(df["Carga Horária"][index].split(" ")[2])
        df["Carga Horária"][index] = df["Carga Horária"][index].split(" ")[
            0].replace("h", "")
        df["Nota"][index] = float(df["Nota"][index])

        if df["Estado"][index] == "APROVADO" or "REPROVADO" in df["Estado"][index] and df["Período"][index] != "2019/2":
            cr += df["Nota"][index]*df["Créditos"][index]
            dem += df["Créditos"][index]

        try:
            df["Frequência"][index] = float(
                df["Frequência"][index].replace("%", "")) / 100
        except:
            df["Frequência"][index] = 0.0
            pass

    df = df[["Período", "Disciplina", "Professor", "Carga Horária",
             "Créditos", "Nota", "Frequência", "Estado"]]
    return df[df.Estado != "CANCELADA"], cr/dem


df = site_login(url)
print("finished")

back_df = df.copy()
df, cr = process_df(back_df)
print("CR: %.2f" % cr)
#df.to_csv(r'consulta_acadêmica_Yasmin.csv', index=False)